In [1]:
import torch
import torchvision
import os
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

In [2]:
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = base_dir
        self.transform = transform
        self.data = []
        categories = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

        for idx, category in enumerate(categories):
            path = os.path.join(base_dir, category)
            images = os.listdir(path)[:1000]  # Take only 1000 images per category
            for image in images:
                self.data.append((os.path.join(path, image), idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming 'base_dir' is your dataset directory path
base_dir = 'data/gaussian_filtered_images/gaussian_filtered_images/'  # Update this path
dataset = DiabeticRetinopathyDataset(base_dir=base_dir, transform=transform)

# Splitting the dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:
import torch
import torch.nn as nn
import torchvision.models as models

def modify_efficientnet(model_name, num_classes=5):
    model = getattr(models, model_name)(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(num_features, num_classes)
    )
    return model

In [7]:
def train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, device, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'{model_name} - Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, ' +
              f'Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {100 * correct / total}%')


In [8]:
efficientnet_variants = ['efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 
                         'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 
                         'efficientnet_b6']

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_classes = 5

In [10]:
for model_name in efficientnet_variants:
    print(f"Setting up {model_name}...")
    model = modify_efficientnet(model_name, num_classes)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
    
    train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, device)

Setting up efficientnet_b0...
efficientnet_b0 - Epoch 1, Train Loss: 1.1060106017523341, Validation Loss: 0.8952856030729082, Validation Accuracy: 68.00699300699301%
efficientnet_b0 - Epoch 2, Train Loss: 0.8937635694940885, Validation Loss: 0.8274172130558226, Validation Accuracy: 70.1048951048951%
efficientnet_b0 - Epoch 3, Train Loss: 0.8257279694080353, Validation Loss: 0.806270506646898, Validation Accuracy: 71.15384615384616%
efficientnet_b0 - Epoch 4, Train Loss: 0.8042207091218896, Validation Loss: 0.7905620253748364, Validation Accuracy: 71.5034965034965%
efficientnet_b0 - Epoch 5, Train Loss: 0.7594141082631217, Validation Loss: 0.7861546393897798, Validation Accuracy: 71.5034965034965%
efficientnet_b0 - Epoch 6, Train Loss: 0.7549642879101965, Validation Loss: 0.7787978698809942, Validation Accuracy: 72.55244755244755%
efficientnet_b0 - Epoch 7, Train Loss: 0.7356094643473625, Validation Loss: 0.7735428230630027, Validation Accuracy: 71.85314685314685%
efficientnet_b0 - Epoc

Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-533bc792.pth" to /home/pavan/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-533bc792.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

efficientnet_b1 - Epoch 1, Train Loss: 1.075275090833505, Validation Loss: 0.8847550451755524, Validation Accuracy: 69.05594405594405%
efficientnet_b1 - Epoch 2, Train Loss: 0.877599760890007, Validation Loss: 0.8211860093805525, Validation Accuracy: 70.8041958041958%
efficientnet_b1 - Epoch 3, Train Loss: 0.828384191625648, Validation Loss: 0.7875575323899587, Validation Accuracy: 72.37762237762237%
efficientnet_b1 - Epoch 4, Train Loss: 0.7788025968604617, Validation Loss: 0.791471928358078, Validation Accuracy: 69.93006993006993%
efficientnet_b1 - Epoch 5, Train Loss: 0.752405024237103, Validation Loss: 0.7704993900325563, Validation Accuracy: 71.15384615384616%
efficientnet_b1 - Epoch 6, Train Loss: 0.7362062463329898, Validation Loss: 0.7585216379827924, Validation Accuracy: 71.85314685314685%
efficientnet_b1 - Epoch 7, Train Loss: 0.7338419916729132, Validation Loss: 0.757743678159184, Validation Accuracy: 71.5034965034965%
efficientnet_b1 - Epoch 8, Train Loss: 0.714445792966418

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /home/pavan/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth


...

  0%|          | 0.00/117M [00:00<?, ?B/s]

efficientnet_b5 - Epoch 1, Train Loss: 1.1253985456294484, Validation Loss: 1.1982048153877258, Validation Accuracy: 48.25174825174825%
efficientnet_b5 - Epoch 2, Train Loss: 0.9146766861279806, Validation Loss: 1.0178093877103593, Validation Accuracy: 60.31468531468531%
efficientnet_b5 - Epoch 3, Train Loss: 0.8699418587817086, Validation Loss: 0.8875446187125312, Validation Accuracy: 66.60839160839161%
efficientnet_b5 - Epoch 4, Train Loss: 0.839444734983974, Validation Loss: 0.8388316598203447, Validation Accuracy: 69.75524475524476%
efficientnet_b5 - Epoch 5, Train Loss: 0.7867875488268005, Validation Loss: 0.8154332571559482, Validation Accuracy: 70.1048951048951%
efficientnet_b5 - Epoch 6, Train Loss: 0.7788965267439684, Validation Loss: 0.7991427448060777, Validation Accuracy: 71.85314685314685%
efficientnet_b5 - Epoch 7, Train Loss: 0.7579994768732123, Validation Loss: 0.7896021008491516, Validation Accuracy: 72.37762237762237%
efficientnet_b5 - Epoch 8, Train Loss: 0.732456505

Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-c76e70fd.pth" to /home/pavan/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

efficientnet_b6 - Epoch 1, Train Loss: 1.1523591495222516, Validation Loss: 1.217050290770001, Validation Accuracy: 46.32867132867133%
efficientnet_b6 - Epoch 2, Train Loss: 0.9423360824584961, Validation Loss: 0.9847341212961409, Validation Accuracy: 63.11188811188811%
efficientnet_b6 - Epoch 3, Train Loss: 0.8553327951166365, Validation Loss: 0.876792248752382, Validation Accuracy: 65.73426573426573%
efficientnet_b6 - Epoch 4, Train Loss: 0.8233305219974782, Validation Loss: 0.8236512144406637, Validation Accuracy: 68.88111888111888%
efficientnet_b6 - Epoch 5, Train Loss: 0.8071304199596246, Validation Loss: 0.7943778137365977, Validation Accuracy: 71.32867132867133%
efficientnet_b6 - Epoch 6, Train Loss: 0.7821388012833066, Validation Loss: 0.7718489236301846, Validation Accuracy: 70.8041958041958%
efficientnet_b6 - Epoch 7, Train Loss: 0.7687339633703232, Validation Loss: 0.7715210914611816, Validation Accuracy: 71.85314685314685%
efficientnet_b6 - Epoch 8, Train Loss: 0.7392782237